In [1]:
import geopandas as gpd
import pandas as pd
from google_geocoding import *
from feature_dev import *
import utils

%load_ext autoreload
%autoreload 2

# pd.set_option('display.max_rows', None)

# Driving Travel Times

In [2]:
gdf = load_station_points()
df = construct_ods_for_routing(gdf)

In [3]:
# modes = ['driving']
# query_routes(df, modes)

In [4]:
driving_summaries = construct_driving_summary(df)
driving_summaries.to_csv('google_driving_summaries.csv')

In [5]:
driving_summaries.head()

,station_name_o,lat_o,lon_o,station_name_d,lat_d,lon_d,google_driving_minutes,google_driving_miles
0,Addison Road,38.88706,-76.89581,Anacostia,38.86278,-76.99563,20.05,9.35
193,Archives,38.89389,-77.02193,Anacostia,38.86278,-76.99563,8.98,4.58
289,Arlington Cemetery,38.88449,-77.06338,Anacostia,38.86278,-76.99563,12.92,6.10
385,Ashburn,39.00457,-77.49112,Anacostia,38.86278,-76.99563,39.27,35.45
481,Ballston-MU,38.88216,-77.11180,Anacostia,38.86278,-76.99563,18.97,9.63


In [6]:
len(driving_summaries)

9312

In [7]:
driving_summaries[driving_summaries.duplicated(subset=['station_name_o','station_name_d'], keep=False)]

,station_name_o,lat_o,lon_o,station_name_d,lat_d,lon_d,google_driving_minutes,google_driving_miles


# Bus Travel Times

In [13]:
# gdf = load_station_points()
# dfs = construct_grouped_ods_for_routing(gdf)

# # Pickle groups
# with open('transit_group_dataframes.pickle', 'wb') as handle:
#     pickle.dump(dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
pickle_path = '2024 Data Collection/Google Travel Times/raw data/transit_group_dataframes.pickle'
pickle_path = os.path.join(utils.configs.box_data_dir, pickle_path)
with open(pickle_path, 'rb') as handle:
    dfs = pickle.load(handle)

In [15]:
# modes = [('transit','bus')]
# query_grouped_routes(dfs, modes)

In [20]:
bus_summaries = construct_bus_transit_summary(dfs)
bus_summaries.to_csv('google_transit_bus_summaries.csv')

In [21]:
bus_summaries.head()

,station_name_o,lat_o,lon_o,station_name_d,lat_d,lon_d,google_transit_minutes,google_transit_bus_minutes,google_transit_heavy_rail_minutes,google_transit_subway_minutes,google_transit_tram_minutes,google_transit_walking_minutes,google_transit_waiting_minutes
0,Addison Road,38.88672,-76.89575,Arlington Cemetery,38.88458,-77.06344,40.25,NaN,NaN,35.0,NaN,4.72,0.53
37,Archives,38.89408,-77.02230,Arlington Cemetery,38.88458,-77.06344,31.30,11.13,NaN,7.0,NaN,6.17,7.00
278,Eisenhower Ave,38.80052,-77.07094,Arlington Cemetery,38.88458,-77.06344,32.52,NaN,NaN,19.0,NaN,4.00,9.52
373,Friendship Heights,38.96041,-77.08568,Arlington Cemetery,38.88458,-77.06344,56.12,35.15,NaN,6.0,NaN,10.08,4.88
383,Gallery Place,38.89837,-77.02218,Arlington Cemetery,38.88458,-77.06344,23.67,NaN,NaN,11.0,NaN,11.87,0.80


In [23]:
len(bus_summaries)

9303

In [24]:
bus_summaries[bus_summaries.duplicated(subset=['station_name_o','station_name_d'], keep=False)]


,station_name_o,lat_o,lon_o,station_name_d,lat_d,lon_d,google_transit_minutes,google_transit_bus_minutes,google_transit_heavy_rail_minutes,google_transit_subway_minutes,google_transit_tram_minutes,google_transit_walking_minutes,google_transit_waiting_minutes


# Bus Competativeness with Metro

In [ ]:
metro_tt_path = '2024 Data Collection/WMATA Metro Travel Times/metro_travel_times.csv'
metro_tt_path = os.path.join(utils.configs.box_data_dir, metro_tt_path)  
metro_tt = pd.read_csv(metro_tt_path)
df = calculate_bus_competativeness_index(metro_tt, bus_summaries)
cols = {
    'station_name_o':'station_name_o',
    'station_name_d':'station_name_d',
    'google_transit_minutes':'bus_transit_minutes',
    'metro_minutes':'metro_transit_minutes',
    'bus_competativeness_index':'bus_competativeness_index',
}
df = df[cols.keys()].rename(columns=cols)

df.to_csv('bus_competativeness_index.csv')

In [ ]:
df.sort_values('bus_competativeness_index', ascending=False)

In [29]:
df[df.duplicated(subset=['station_name_o','station_name_d'], keep=False)]

,station_name_o,station_name_d,bus_transit_minutes,metro_transit_minutes,bus_competativeness_index


In [30]:
df

,station_name_o,station_name_d,bus_transit_minutes,metro_transit_minutes,bus_competativeness_index
0,Anacostia,Anacostia,NaN,0,1.000000
1,Anacostia,Archives,34.02,9,0.264550
2,Anacostia,Benning Road,54.20,26,0.479705
3,Anacostia,Brookland-CUA,65.25,27,0.413793
4,Anacostia,Capitol South,28.82,16,0.555170
...,...,...,...,...,...
9404,Ashburn,Herndon,20.85,17,0.815348
9405,Ashburn,Innovation Center,22.40,14,0.625000
9406,Ashburn,Dulles Airport,11.25,10,0.888889
9407,Ashburn,Loudoun Gateway,7.38,4,0.542005
